# Tables Basic

In [2]:
# basic setup
import datetime
from dateutil.relativedelta import relativedelta
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
from pathlib import Path
import pytz
import statsmodels.api as sm
import statsmodels.formula.api as smf
import string
import sys

# function to create a tabular table
sys.path.append('submodules/python-tabular-output/')
from tab_general_func import tabularconvert
from tab_general_func import mergetabsecs

## Basic Example

In [3]:
tabular = tabularconvert([['Col1', 'Col2'], ['a', 'b'], ['1', '2']], colalign = '|l|r|', hlines = [0, 1, -1], savename = None)
print(tabular)

\begin{tabular}{|l|r|}
\hline
Col1 & Col2 \\
\hline
a & b \\
1 & 2 \\
\hline
\end{tabular}



## Multicolumn

In [5]:
tabular = tabularconvert([['\\multicolumn{2}{|c|}{Cols. 1 and 2}', 'Col3'], ['Col1', 'Col2', 'Col3'], ['a', 'b', 'c'], ['1', '2', '3']], colalign = '|c|c|c|', hlines = [0, 1, 2, -1], savename = None)
print(tabular)

\begin{tabular}{|c|c|c|}
\hline
\multicolumn{2}{|c|}{Cols. 1 and 2} & Col3 \\
\hline
Col1 & Col2 & Col3 \\
\hline
a & b & c \\
1 & 2 & 3 \\
\hline
\end{tabular}



## Multirow

In [7]:
tabular = tabularconvert([['', 'Col1', 'Col2'], ['\\multirow{2}{*}{Letters}', 'a', 'b'], ['', 'A', 'B'], ['Numbers', '1', '2']], colalign = 'lcc', hlines = [0, 1, 3, -1], savename = None)
print(tabular)

\begin{tabular}{lcc}
\hline
 & Col1 & Col2 \\
\hline
\multirow{2}{*}{Letters} & a & b \\
 & A & B \\
\hline
Numbers & 1 & 2 \\
\hline
\end{tabular}



## Regression Example

In [8]:
# data setup:{{{
N = 1000
Nfirsthalf = N // 2
beta1 = 1
beta2 = 1

x1 = np.random.normal(size = N)
x2 = np.random.normal(size = N)
epsilon = np.random.normal(size = N)

df = pd.DataFrame({'x1': x1, 'x2': x2, 'epsilon': epsilon})
df['y1'] = beta1 * df['x1'] + beta2 * df['x2'] + df['epsilon']
df['y2'] = 2 * beta1 * df['x1'] + 2 * beta2 * df['x2'] + 2 * df['epsilon']
# data setup:}}}

outputrows = []
# add title row
outputrows.append(['x/y', 'y1', 'y2'])

# put x in rows and y in columns
for x in ['x1', 'x2']:
    betarow = [x]
    stdnrow = ['']
    for y in ['y1', 'y2']:
        df2 = df[(df[y].notna()) & (df[x].notna())].copy()

        model = smf.ols(formula = y + ' ~ ' + x, data = df2).fit()

        beta = model.params[1]
        pval = model.pvalues[1]
        stderr = model.bse[1]
        # tstat = model.tvalues[1]
        n = len(df2)

        # add stars
        abeta = '{:.3f}'.format(beta)
        if pval < 0.05:
            abeta = abeta + '*'
        if pval < 0.01:
            abeta = abeta + '*'
        if pval < 0.001:
            abeta = abeta + '*'

        stderr = '{:.3f}'.format(stderr)
        n = str(n)

        betarow.append(abeta)
        stdnrow.append('(' + stderr + ',' + n + ')')

    outputrows.append(betarow)
    outputrows.append(stdnrow)

tabular = tabularconvert(outputrows, colalign = 'lcc', hlines = [0, 1, -1], savename = None)
print(tabular)

\begin{tabular}{lcc}
\hline
x/y & y1 & y2 \\
\hline
x1 & 1.032*** & 2.063*** \\
 & (0.046,1000) & (0.092,1000) \\
x2 & 0.984*** & 1.968*** \\
 & (0.046,1000) & (0.091,1000) \\
\hline
\end{tabular}

